In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset Loading and Transformation
# Data Augmentation and Normalization for Training, Normalization for Testing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the images on the horizontal
    transforms.RandomRotation(10),      # Random rotation of the images by +/- 10 degrees
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=test_transform, download=True)

# Hyperparameters
batch_size = 256  # Example batch size, can be tuned
learning_rate = 0.001  # Example learning rate, can be tuned
num_epochs = 105  # Example number of epochs, can be tuned

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

class IntermediateBlock(nn.Module):
    def __init__(self, in_channels, num_layers, num_channels, dropout_prob=0.5):
        super(IntermediateBlock, self).__init__()
        self.layers = nn.ModuleList([])
        for _ in range(num_layers):
            self.layers.append(nn.Sequential(
                nn.Conv2d(in_channels, num_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(num_channels),
                nn.Dropout(dropout_prob),
                nn.ReLU()
            ))
        self.fc = nn.Linear(in_channels, num_layers)

    def forward(self, x):
        m = x.mean([2, 3])  # Global average pooling
        a = torch.sigmoid(self.fc(m))  # Compute a vector
        outputs = [layer(x) for layer in self.layers]
        a = a.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)  # Reshape a to [batch_size, num_layers, 1, 1, 1]
        outputs = torch.stack(outputs, dim=1)  # Shape [batch_size, num_layers, num_channels, H, W]
        x_prime = (a * outputs).sum(dim=1)  # Sum across the layer dimension
        return x_prime


class BasicArch(nn.Module):
    def __init__(self):
        super(BasicArch, self).__init__()
        self.block1 = IntermediateBlock(3, 3, 64)
        self.block2 = IntermediateBlock(64, 3, 128)
        self.block3 = IntermediateBlock(128, 3, 256)
        self.block4 = IntermediateBlock(256, 3, 256)
        self.fc = nn.Linear(256, 10)  # Final layer

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = x.mean([2, 3])  # Global average pooling before the final layer
        logits = self.fc(x)
        return logits

model = BasicArch().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(model, train_loader, test_loader, num_epochs, optimizer, criterion):
    train_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = 100 * correct_train / total_train

        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        test_accuracy = test()
        test_accuracies.append(test_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')

    return train_losses, train_accuracies, test_accuracies


def test():
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_acc = 100 * correct_test / total_test
    return test_acc

train(model, train_loader, test_loader, num_epochs, optimizer, criterion)


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/105], Loss: 1.5869, Training Accuracy: 41.27%, Test Accuracy: 47.46%
Epoch [2/105], Loss: 1.2657, Training Accuracy: 54.52%, Test Accuracy: 51.85%
Epoch [3/105], Loss: 1.1452, Training Accuracy: 59.07%, Test Accuracy: 54.76%
Epoch [4/105], Loss: 1.0619, Training Accuracy: 62.13%, Test Accuracy: 61.47%
Epoch [5/105], Loss: 0.9927, Training Accuracy: 64.97%, Test Accuracy: 55.38%
Epoch [6/105], Loss: 0.9399, Training Accuracy: 66.75%, Test Accuracy: 61.24%
Epoch [7/105], Loss: 0.8961, Training Accuracy: 68.39%, Test Accuracy: 59.13%
Epoch [8/105], Loss: 0.8654, Training Accuracy: 69.64%, Test Accuracy: 48.40%
Epoch [9/105], Loss: 0.8345, Training Accuracy: 70.44%, Test Accuracy: 59.04%
Epoch [10/105], Loss: 0.8038, Training Accuracy: 71.73%, Test Accuracy: 57.06%
Epoch [11/105], Loss: 0.7736, Training Accuracy: 72.98%, Test Accuracy: 59.53%
Epoch [12/105], Loss: 0.7546, Training Accuracy: 73.61%, Test Ac

([1.586879220199585,
  1.2656749743652345,
  1.1451724950790405,
  1.0619453915023804,
  0.9926983252716064,
  0.9399130902099609,
  0.8961188280868531,
  0.8653735596466064,
  0.8345277185058594,
  0.8038349489593506,
  0.7735565811920166,
  0.7546350347518921,
  0.732029234008789,
  0.7129351515388489,
  0.6994860038375854,
  0.681779351978302,
  0.6661499557685852,
  0.6521569897079468,
  0.638541390991211,
  0.6265782439613342,
  0.611850696105957,
  0.603482518119812,
  0.5934302132606506,
  0.5828479667854309,
  0.5724204182243348,
  0.5634015159988404,
  0.5572007258987427,
  0.5456436315917969,
  0.5400920338058471,
  0.5285150450134277,
  0.5221635419464111,
  0.5128871480941772,
  0.5056756968307495,
  0.49885956563949585,
  0.4893455916404724,
  0.48758629596710207,
  0.4774433709049225,
  0.4707290411376953,
  0.4633014723110199,
  0.4580940888595581,
  0.45201127361297605,
  0.45133315628051757,
  0.4409128058719635,
  0.43816575159072874,
  0.43174622934341433,
  0.422872

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, 'ro-', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, 'bo-', label='Training Accuracy')
plt.plot(test_accuracies, 'go-', label='Test Accuracy')
plt.title('Training and Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# plot_metrics(train_losses, train_accuracies, test_accuracies)

# New Section